In [2]:
pip install lxml

ERROR: Invalid requirement: 'lxml,BeautifulSoup4'
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install BeautifulSoup4

     |████████████████████████████████| 128 kB 4.6 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [41]:
import time
from csv import DictWriter
from threading import Thread
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Queue, Process
import os
import sys
import re

import requests
import lxml
from lxml import etree
from bs4 import BeautifulSoup

In [83]:
class GEO_crawler:
    def __init__(self, GEO_id):
        self.GEO_id = GEO_id
        self.base_url = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi"
        
    def progress_bar(self, num, total):
        rate = float(num)/total
        #print("rate:%s"%rate)
        ratenum = int(rate * 100)
        #print(ratenum)
        
        show = "\r[{}{}]{}".format("*" * ratenum, "-" * (total - ratenum), str(ratenum) + "%")
        
        if ratenum == 100:
            sys.stdout.write(show)
            sys.stdout.write("\n{} is over\n".format(self.GEO_id))
        else:
            sys.stdout.write(show)
            sys.stdout.flush()
        
        
        
    def item2csv(self, item):
       
            has_header = os.path.exists(self.GEO_id+".csv")

            with open(self.GEO_id +".csv", "a") as file:
                writer = DictWriter(file, list(item.keys()))
                if not has_header:
                    writer.writeheader()
                    has_header = True

                writer.writerow(item)
        
    def download(self):
        resp = requests.get(self.base_url, params={"acc": self.GEO_id})
        if resp.status_code == 200:
            resp.encoding = "utf-8"
            self.parse(resp.text)
    
    def parse(self, html):
        root = BeautifulSoup(html, "lxml")
        sites = root.select("a")
        
        url_ls = []
        
        number = 1
        total = 0
        
        for site in sites:
            text = site.get_text()
            if re.findall("^GSM[0-9]+", text):
                total +=1
        
        for site in sites:
            text = site.get_text()
            if text.startswith("GSM"):
                resp = requests.get(self.base_url, params={"acc": text})
                if resp.status_code == 200:
                    resp.encoding = "utf-8"
                    
                    root = BeautifulSoup(resp.text, "lxml")
                    tags = root.select("tr[valign = top]")
                    
                    item = {}
                    for num,tag in enumerate(tags):
                        if num == 0:
                            continue
                        else:
                            tag_ls = tag.select("td")
                            
                            if len(tag_ls) > 1:
                                if tag_ls[0].get_text() != "\xa0":
                                    item[tag_ls[0].get_text()] = tag_ls[1].get_text()
                            # else:
                            #     if tag_ls[0].get_text() != "\xa0":
                            #         text_ls = []
                            #         for i in range(len(tag_ls)):
                            #             text_ls.append(tag_ls[i].get_text())
                            #         item[text_ls[0]] = 
                    
                    #print(item)
                    self.item2csv(item)
                    self.progress_bar(num = number, total = total)
                    number += 1
    
                
                
            

In [82]:
a = GEO_crawler("GSE201369")
a.download()

[****************************************************************************************************]100%
GSE201369 is over


In [86]:
def main(max_workers = 4, GEO_id_ls = None):
    with ThreadPoolExecutor(max_workers=max_workers) as pool:
        start = time.time()
        for GEO_id in GEO_id_ls:
            if os.path.exists(GEO_id + ".csv"):
                os.remove(GEO_id + ".csv")
            pool.submit(GEO_crawler(GEO_id).download)
    end = time.time()
    print(f"consuming: {end -start:.3f} seconds")

In [88]:
if __name__ == "__main__":
    main(GEO_id_ls=["GSE201369", "GSE44801"])

[****************************************************************************************************]100%
GSE44801 is over
[****************************************************************************************************]100%
GSE201369 is over
consuming: 2.660 seconds
